<a href="https://colab.research.google.com/github/Prisci723/InteligenciaArtificial1/blob/main/One%20vs%20All/Lab3_OnevsAll_Letter_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación multiclase
El siguiente dataset tiene como propósito entrenar de regresión logística (clasificación multiclase) utilizando el cuadernillo que avanzamos en clases, one vs. all.

# Dataset
El siguiente dataset posee de los datos relacionados a la identificación de las letras mayúsculas del alfabeto ingés.

El siguiente dataset está compuesto por 16 features los cuales son:
* x1: posicion de la x
* x2: posicion en y
* x3: ancho del cuadrado
* x4: altura del cuadrado
* x5: total de pixeles
* x6: media de los pixeles en x
* x7: media de los pixeles en y
* x8: varianza de x
* x9: varianza de y
* x10: varianza de la correlación de xy
* x11: media de x*x*y
* x12: media de x*y*y
* x13: media del borde contado de izquierda a derecha
* x14: correlacion del eje x con y
* x15: media del borde de y del fondo al tope
* x16: correlacion del eje y con x


y el target:
* y: que letra es (A-Z)

El dataset fue obtenido de la página:
https://archive.ics.uci.edu/dataset/59/letter+recognition


# Tratamiento con pandas
Como se puede observar el dataset viene en un archivo .csv por defecto el cual posee directamente todos los valores de los features con el target correspondiente, sin tener etiquetas en las columnas, por lo que manualmente, para facilitar el tratamiento es que se añadió una fila con los nombres de cada una.
También se puede comprobar que los datos están completos y no requieren de un tratamiento profundo, simplemente el target, el cual posee datos de tipo caracter, por lo que se procede a cambiar las letras por numeros.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
#pandas para tratar los datos y OrdinalEncoder para transformar los caracteres en numeros

In [ ]:
dataframe = pd.read_csv("/content/drive/MyDrive/Machine learning/Datasets/letter_regonition.csv", sep=",")
print(dataframe)

Una vez que las librerías han sido cargadas, es que se procede a utilizarlas, lo que ocurre es que la columna de target 'y', los caracteres serán reemplazados por numeros de forma ordenada, es decir, que se tendrá el siguiente resultado.



*   A --> 0
*   B --> 1
*  C --> 2
* D --> 3
* E --> 4
* E --> 4
* F --> 5
* G --> 6
* H --> 7
* I --> 8
* J --> 9
* K --> 10
* L --> 11
* M  --> 12
* N --> 13
* O  --> 14
* P  --> 15
* Q  --> 16
* R  --> 17
* S  --> 18
* T  --> 19
* U  --> 20
* V  --> 21
* W  --> 22
* X  --> 23
* Y  --> 24
* Z --> 25






In [ ]:
enc = OrdinalEncoder()
dataframe['y'] = enc.fit_transform(dataframe[['y']])
print(dataframe)

Podemos comprobar que los datos de y, target, ahora son numericos y sevirán para ser aplicados dentro del código

In [ ]:
dataframe['y'].unique()

Procedemos a guardar el archivo modificado

In [ ]:
dataframe.to_csv("letter_recognition.csv")

# Aplicacion de one vs all

El siguiente codigo tiene como objetivo entrenar con los datos correspondientes del dataset, aplicado al codigo de one vs all, el cual consiste en que los datos son separados utilizando clasificadores binarios según cada clase que exista, haciendo que cada clase se distinga de las demás. Al final se obiente la predicción de a que clase pertenece un determinada entrada de X.

In [26]:
import os

import numpy as np

from matplotlib import pyplot

from scipy import optimize

%matplotlib inline

El dataset posee 20000 datos los cuales el 80% es decir 16000 datos serán destinados al entrenamiento. 4000 datos para la pruba.

In [45]:
# 17 elementos contando x0
input_layer_size  = 17
# de 0 a 25
num_labels = 26

#Se carga el dataset
data = np.loadtxt('/content/drive/MyDrive/Machine learning/Datasets/Clean_letter_recognition.csv', delimiter=',')

Y = data[:, 0]  # Primera columna
X = data[:, 1:]  # Resto de las columnas
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, y_train = X[:16000], Y[:16000]
X_test, y_test = X[16000:], Y[16000:]
X = X_train
Y = y_train

m = Y.size

print(X)
print(Y)

[[ 2.   8.   3.  ...  8.3  0.2  8.4]
 [ 5.  12.   3.  ...  8.   4.  10. ]
 [ 4.  11.   6.  ...  7.   3.   9. ]
 ...
 [ 8.  14.   7.  ...  7.   5.   8. ]
 [ 4.   7.   5.  ...  8.   5.   8. ]
 [ 2.   1.   3.  ...  9.   4.  10. ]]
[19.  8.  3. ...  6.  4.  2.]


In [46]:
print(X_test)
print(y_test)

[[ 4. 10.  6. ...  8.  5.  6.]
 [ 6.  9.  8. ...  8.  0.  8.]
 [ 6.  9.  8. ...  9.  7. 10.]
 ...
 [ 6.  9.  6. ... 12.  2.  4.]
 [ 2.  3.  4. ...  9.  5.  8.]
 [ 4.  9.  6. ...  7.  2.  8.]]
[20. 13. 21. ... 19. 18.  0.]


**Normalización de los datos**

Los datos deben ser normalizados para estar dentro de la misma escala, la forma con la que se normalizan los datos es utilizando la media y sigma.

In [47]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [48]:
X_norm, mu, sigma = featureNormalize(X)

X es asignada con los valores normalizados

In [49]:
m, n = X.shape
X = X_norm

**Definicion de la funcion de la sigmoide**

La sigmoide es utilizada para colocar cualquier valor dentro de un rango de 0 y 1, el cual servirá como resultado para calcular las probabilidades de cada clase.

In [50]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

**Calculo de la funcion de cosoo y gradiente para regresión logística**

El objetivo es encontrar los valores más óptimos para cada theta.

Una vez que la funcion sea ejecutada se contará con los valores optimizados de theta.

In [51]:
def lrCostFunction(theta, X, y, lambda_):
    m = y.size

    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

**Función one vs all**

En esta función es donde se entrenerá a las clases, de forma individual cada clase será entrenada para distinguirse de las demás clases.

In [52]:
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape

    all_theta = np.zeros((num_labels, n+1))

    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n+1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [53]:
print(X.shape)

(16000, 16)


In [54]:
lambda_ = 0.1
all_theta = oneVsAll(X, Y, num_labels, lambda_)
print(all_theta.shape)

(26, 17)


In [55]:
print(all_theta)

[[-8.15738409e+00 -1.45686869e+00  4.74686010e-01  1.98509910e+00
   1.42874086e-01 -1.14743524e+00  1.11030096e+00 -7.87220584e-01
  -1.49043784e+00 -2.49800845e+00 -1.50399328e+00 -9.80607082e-01
   1.29512931e+00 -5.29533274e-01 -3.00565543e-01  5.94999541e-01
  -1.09081970e+00]
 [-6.17634334e+00  4.25906625e-02  2.40001686e-01 -1.16094275e+00
  -1.71257276e+00  1.87251095e+00  1.31212284e-01 -9.48085457e-01
   7.56271638e-01  4.95821464e-01  7.90256334e-01  3.97288762e-01
  -1.89824922e+00 -2.17590046e-01 -5.34944069e-01  1.47571451e+00
   9.24428243e-01]
 [-6.58899649e+00  4.18114808e-01  5.97521118e-01 -7.33799351e-01
  -1.00902036e+00  3.10999351e-01  2.13384870e-01  1.16837232e+00
   9.87285526e-01  1.04164462e+00  1.21116382e+00  6.38835213e-01
   2.17994491e+00  7.49836039e-01  7.39220235e-01 -6.88673114e-02
   1.00892353e+00]
 [-7.52299315e+00 -4.63337578e-03  7.59814582e-01 -1.85378865e+00
  -2.17147494e+00  2.72847203e+00 -1.25699830e-03 -1.44798883e+00
   2.13256198e+00  

# Predicciones

Con las funciones anteriores es que ya tenemos los valores de theta que servirán para las prediciciones, al igual que los valores normalizados de X

**Funcion para predecir one vs all**

Para cada ejemplo en X la función calculará la probabilidad de que el ejemplo pertenezca a una clase, esto ocurre gracias a aplicar la regresión logística a la hipótesis, el resultado es una prabilidad enrtre 0 y 1, después la función escogerá la clase a la que la probabilidad es más alta, siendo esta la clase la clase predecida.

In [56]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

**Resultados del entrenamiento**

Ahora es que se llama a los valores destinados a mostar la prueba del entrenamiento, X_test y Y_test, mostrando también la precisión de los datos predecidos.

**Normalizacion de los valores de X_test**

Previo al entreanamiento los valores de X_test deben ser normalizados

In [57]:
X_test_norm, mu_test, sigma_test = featureNormalize(X_test)

In [58]:
X_test = X_test_norm

Para visualizar los resultados es que solo se imprime las filas de 10 a 150

In [59]:
print(X_test.shape)
pred = predictOneVsAll(all_theta, X_test)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y_test) * 100))
XPrueba = X_test.copy()
print(XPrueba.shape)

m, n = X_test.shape

XPrueba = np.concatenate([np.ones((m, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p[10:150])


print(y_test[10:150])

(4000, 16)
Precision del conjuto de entrenamiento: 71.55%
(4000, 16)
(4000, 17)
[13  8  6 11 17  1 12  3 21 17 15  3  4 22  3 18 17  6 21 17  3  9  6  8
  6  2 13 12  1 13 17  0  0 16  1  7 17  2 18  0  9 10 22 13  7 18  1  4
 24  9 13 16 24 16 23 19 11  6 15 10 22  7 22  0 13 19  8  5  8  3 19  1
 19  8  3  5  2 18  2  1  0 13 10 17  7 11 15  2 13  5  8 25  5  3  6  1
 25 19 16 21 17 20 22  1  9 17 10 16  1 17  5 18 16 11 19 15  0 10  5  3
 16 11 16  2 13 10  8 15 17  3 20 21 12  0 17 24 10  6 23 11]
[13.  1.  6. 11.  4.  6. 12.  3. 24. 17. 15.  3.  4. 22.  3. 16. 17.  6.
 24. 17.  3.  9.  2.  8. 19.  2. 13. 12.  1.  7. 17. 16.  0.  6. 16.  7.
 10.  6.  0.  0.  9. 10. 22. 13.  7. 11.  1.  4. 24.  9. 13. 11. 18. 16.
  3.  4. 11.  6. 15. 10. 22. 14. 22.  0.  7. 19.  8.  5.  8.  3. 19. 15.
 25.  8.  3.  5.  6. 19.  2.  1.  7. 13.  4. 17. 20. 11. 15.  2. 13.  5.
  8. 25.  5.  3.  6.  1. 18. 19. 16. 21.  4.  0. 22. 10. 24. 10. 11. 16.
  1. 18.  5. 18. 16. 11. 19. 15.  0. 10.  5.  3. 16. 11